In [1]:
import random

# Encoding

In [2]:
Q = 41

In [3]:
def encode(x):
    return x % Q

def decode(x):
    return x if x <= Q/2 else x-Q

In [4]:
x = encode(-5)
print("encoded: %d" % x)
print("decoded: %d" % decode(x))

encoded: 36
decoded: -5


# Additive sharing

In [5]:
N = 10

In [6]:
def additive_share(secret):
    shares  = [ random.randrange(Q) for _ in range(N-1) ]
    shares += [ (secret - sum(shares)) % Q ]
    return shares

def additive_reconstruct(shares):
    return sum(shares) % Q

shares = additive_share(5)
print(shares)
print(additive_reconstruct(shares))

[37, 23, 36, 26, 20, 14, 28, 40, 14, 13]
5


In [7]:
def explain(seen_shares, guess):
    # compute the unseen share that justifies the seen shares and the guess
    simulated_unseen_share = (guess - sum(seen_shares)) % Q
    # and would-be sharing by combining seen and unseen shares
    simulated_shares = seen_shares + [simulated_unseen_share]
    if additive_reconstruct(simulated_shares) == guess:
        # found an explanation
        return simulated_unseen_share

seen_shares = shares[:N-1]
for guess in range(Q):
    explanation = explain(seen_shares, guess)
    if explanation is not None: 
        print("guess %d can be explained by %d" % (guess, explanation))

guess 0 can be explained by 8
guess 1 can be explained by 9
guess 2 can be explained by 10
guess 3 can be explained by 11
guess 4 can be explained by 12
guess 5 can be explained by 13
guess 6 can be explained by 14
guess 7 can be explained by 15
guess 8 can be explained by 16
guess 9 can be explained by 17
guess 10 can be explained by 18
guess 11 can be explained by 19
guess 12 can be explained by 20
guess 13 can be explained by 21
guess 14 can be explained by 22
guess 15 can be explained by 23
guess 16 can be explained by 24
guess 17 can be explained by 25
guess 18 can be explained by 26
guess 19 can be explained by 27
guess 20 can be explained by 28
guess 21 can be explained by 29
guess 22 can be explained by 30
guess 23 can be explained by 31
guess 24 can be explained by 32
guess 25 can be explained by 33
guess 26 can be explained by 34
guess 27 can be explained by 35
guess 28 can be explained by 36
guess 29 can be explained by 37
guess 30 can be explained by 38
guess 31 can be expl

In [8]:
def additive_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def additive_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [9]:
class Additive:
    
    def __init__(self, secret=None):
        self.shares = additive_share(encode(secret)) if secret is not None else []
    
    def reveal(self):
        return decode(additive_reconstruct(self.shares))
    
    def __repr__(self):
        return "Additive(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Additive()
        z.shares = additive_add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = Additive()
        z.shares = additive_sub(x.shares, y.shares)
        return z

x = Additive(5)
print(x)

y = Additive(8)
print(y)

z = x - y
print(z)
assert(z.reveal() == -3)

Additive(5)
Additive(8)
Additive(-3)


# Polynomials

In [10]:
# using Horner's rule

def evaluate_at_point(coefs, point):
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % Q
    return result

In [11]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# from http://www.ucl.ac.uk/~ucahcjm/combopt/ext_gcd_python_programs.pdf
def egcd_binary(a,b):
    u, v, s, t, r = 1, 0, 0, 1, 0
    while (a % 2 == 0) and (b % 2 == 0):
        a, b, r = a//2, b//2, r+1
    alpha, beta = a, b
    while (a % 2 == 0):
        a = a//2
        if (u % 2 == 0) and (v % 2 == 0):
            u, v = u//2, v//2
        else:
            u, v = (u + beta)//2, (v - alpha)//2
    while a != b:
        if (b % 2 == 0):
            b = b//2
            if (s % 2 == 0) and (t % 2 == 0):
                s, t = s//2, t//2
            else:
                s, t = (s + beta)//2, (t - alpha)//2
        elif b < a:
            a, b, u, v, s, t = b, a, s, t, u, v
        else:
            b, s, t = b - a, s - u, t - v
    return (2 ** r) * a, s, t


def inverse(a):
    _, b, _ = egcd_binary(a, Q)
    return b

In [12]:
# see https://en.wikipedia.org/wiki/Lagrange_polynomial

def lagrange_constants_for_point(points, point):
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - point)) % Q
                denum = (denum * (xj - xi)) % Q
        constants[i] = (num * inverse(denum)) % Q
    return constants

def interpolate_at_point(points_values, point):
    points, values = zip(*points_values)
    constants = lagrange_constants_for_point(points, point)
    return sum( vi * ci for vi, ci in zip(values, constants) ) % Q

# Shamir sharing

In [13]:
N = 10
T = 4

assert(T+1 <= N)

In [14]:
def sample_shamir_polynomial(zero_value):
    coefs = [zero_value] + [random.randrange(Q) for _ in range(T)]
    return coefs

In [15]:
SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(0 not in SHARE_POINTS)

def shamir_share(secret):
    polynomial = sample_shamir_polynomial(secret)
    shares = [ evaluate_at_point(polynomial, p) for p in SHARE_POINTS ]
    return shares

def shamir_reconstruct(shares):
    polynomial = [ (p,v) for p,v in zip(SHARE_POINTS, shares) if v is not None ]
    secret = interpolate_at_point(polynomial, 0)
    return secret

shares = shamir_share(5)
for i in range(N-(T+1)):
    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
x = shamir_reconstruct(shares)
assert(x == 5)

In [16]:
def shamir_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def shamir_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [17]:
def shamir_mul(x, y):
    return [ (xi * yi) % Q for xi, yi in zip(x, y) ]

In [18]:
class Shamir:
    
    def __init__(self, secret=None):
        self.shares = shamir_share(encode(secret)) if secret is not None else []
        self.degree = T
    
    def reveal(self):
        assert(self.degree+1 <= N)
        return decode(shamir_reconstruct(self.shares))
    
    def __repr__(self):
        return "Shamir(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Shamir()
        z.shares = shamir_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Shamir()
        z.shares = shamir_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Shamir()
        z.shares = shamir_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z
    
x = Shamir(2)
print(x)

y = Shamir(3)
print(y)

z = x - y
print(z)
assert(z.reveal() == -1)

v = x * y
print(v)
assert(v.reveal() == 6)

Shamir(2)
Shamir(3)
Shamir(-1)
Shamir(6)


# Packed sharing

In [19]:
N = 20
T = 8
K = 2

assert(T+K <= N)

In [20]:
SECRET_POINTS =     [ -p % Q for p in range(1, K+1) ]
RANDOMNESS_POINTS = [ -p % Q for p in range(K+1, K+T+1) ]
assert(set(SECRET_POINTS).intersection(RANDOMNESS_POINTS) == set())

def sample_packed_polynomial(secrets):
    assert(len(secrets) == K)
    points = SECRET_POINTS + RANDOMNESS_POINTS
    values = secrets + [ random.randrange(Q) for _ in range(T) ]
    return list(zip(points, values))

In [21]:
SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(set(SHARE_POINTS).intersection(SECRET_POINTS) == set())
assert(set(SHARE_POINTS).intersection(RANDOMNESS_POINTS) == set())

def packed_share(secrets):
    polynomial = sample_packed_polynomial(secrets)
    shares = [ interpolate_at_point(polynomial, p) for p in SHARE_POINTS ]
    return shares

def packed_reconstruct(shares):
    points = SHARE_POINTS
    values = shares
    polynomial = [ (p,v) for p,v in zip(points, values) if v is not None ]
    secrets = [ interpolate_at_point(polynomial, p) for p in SECRET_POINTS ]
    return secrets

secrets = [5,6]
shares = packed_share(secrets)
for i in range(N-(T+K)):
    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
reconstructed_secrets = packed_reconstruct(shares)
assert(reconstructed_secrets == secrets)

In [22]:
def packed_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def packed_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [23]:
def packed_mul(x, y):
    return [ (xi * yi) % Q for xi, yi in zip(x, y) ]

In [24]:
class Packed:
    
    def __init__(self, secrets=None):
        self.shares = packed_share([ encode(s) for s in secrets ]) if secrets is not None else []
        self.degree = T+K-1
    
    def reveal(self):
        assert(self.degree+1 <= N)
        #print(packed_reconstruct(self.shares))
        return [ decode(s) for s in packed_reconstruct(self.shares) ]
    
    def __repr__(self):
        return "Packed(%s)" % self.reveal()
    
    def __add__(x, y):
        z = Packed()
        z.shares = packed_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Packed()
        z.shares = packed_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Packed()
        z.shares = packed_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z
    
x = Packed([2,3])
print(x)

y = Packed([2,3])
print(y)

z = x - y
print(z)
assert(z.reveal() == [0,0])

v = x * y
print(v)
assert(v.reveal() == [4,9])

Packed([2, 3])
Packed([2, 3])
Packed([0, 0])
Packed([4, 9])
